In [ ]:
import pandas as pd
from datetime import timedelta

# نصب کتابخانه tbats اگر نصب نیست (در محیط خود کاربر اجرا شود)
# !pip install tbats

from tbats import TBATS

# خواندن فایل ورودی
df = pd.read_csv('merged_output2.csv')

# تبدیل DATE_MILADI و HOUR به datetime
df['DATE_MILADI'] = pd.to_datetime(df['DATE_MILADI'])
df['datetime'] = df['DATE_MILADI'] + pd.to_timedelta(df['HOUR'], unit='h')

# مرتب‌سازی بر اساس datetime
df = df.sort_values('datetime').reset_index(drop=True)

# چک کردن duplicate در datetime
if df['datetime'].duplicated().any():
    print("هشدار: datetimeهای تکراری وجود دارد. در حال aggregation با میانگین POWER...")
    df = df.groupby('datetime').agg({
        'POWER': 'mean',
        'DATE_MILADI': 'first',
        'HOUR': 'first',
        'DATE_SHAMSI': 'first',
        'CODE': 'first',
        'UNIT_NO': 'first',
        'DAMA': 'mean',
        'ROTOOBAT': 'mean',
        '12209_G13': 'mean',
        '12210_G13': 'mean',
        'ebraz': 'first',
        'importance_factor': 'first'
    }).reset_index()

# استخراج year, month, dayofweek
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['dayofweek'] = df['datetime'].dt.dayofweek

# فیلتر داده آموزشی (2022 تا 2023) و تست (از 2024 به بعد)
train_mask = df['year'].between(2022, 2023)
test_mask = df['year'] >= 2024

train = df[train_mask]
test = df[test_mask].copy()

# ایجاد سری زمانی POWER با index datetime
power_series_full = df.set_index('datetime')['POWER']

# تنظیم فرکانس ساعتی با forward fill
power_series_full = power_series_full.asfreq('h', method='ffill')

# اضافه کردن ستون DECLARED به تست
test['DECLARED'] = 0.0

# لوپ روی ردیف‌های تست برای پیش‌بینی سه روز آینده با TBATS
for idx, row in test.iterrows():
    current_datetime = row['datetime']
    
    if row['ebraz'] == 0:
        test.at[idx, 'DECLARED'] = 0
        continue
    
    # زمان پایه برای آموزش: دقیقاً سه روز قبل
    train_end_time = current_datetime - timedelta(days=3)
    
    # داده‌های موجود تا train_end_time
    train_data = power_series_full[power_series_full.index <= train_end_time]
    
    if len(train_data) < 168:  # حداقل یک هفته برای فصل
        test.at[idx, 'DECLARED'] = 0
        continue
    
    # چک کردن مقادیر صفر/منفی (TBATS نیاز به داده مثبت دارد اگر Box-Cox استفاده شود)
    if (train_data <= 0).any():
        print(f"مقدار صفر/منفی در داده آموزشی برای {current_datetime} — پر کردن با مقدار کوچک مثبت")
        train_data = train_data.clip(lower=1e-6)  # جایگزین صفر با مقدار کوچک
    
    # فیت مدل TBATS با فصل روزانه (24 ساعته)
    # می‌توانید seasonal_periods=[24, 168] برای روزانه و هفتگی اضافه کنید اگر داده اجازه دهد
    model = TBATS(
        seasonal_periods=[24],  # فصل روزانه
        use_box_cox=True,       # Box-Cox transformation
        use_trend=True,         # Trend
        use_damped_trend=True,  # Damped trend
        use_arma_errors=True    # ARMA errors
    ).fit(train_data)
    
    # پیش‌بینی دقیقاً 72 گام جلو (3 روز)
    forecast = model.forecast(steps=72)
    
    # مقدار پیش‌بینی برای ساعت مربوطه
    predicted_value = forecast[-1]
    
    # محدود به صفر اگر منفی
    test.at[idx, 'DECLARED'] = max(0.0, predicted_value)

# ستون‌های خروجی
output_cols = [
    'HOUR', 'DATE_MILADI', 'DATE_SHAMSI', 'POWER', 'CODE', 'UNIT_NO',
    'DAMA', 'ROTOOBAT', '12209_G13', '12210_G13', 'ebraz', 'importance_factor',
    'year', 'month', 'dayofweek', 'DECLARED'
]

# ذخیره خروجی
test[output_cols].to_csv('output_tbats.csv', index=False)

print("فایل خروجی output_tbats.csv با موفقیت ایجاد شد.")
print(f"تعداد ردیف‌های پیش‌بینی‌شده: {len(test)}")

هشدار: datetimeهای تکراری وجود دارد. در حال aggregation با میانگین POWER...
مقدار صفر/منفی در داده آموزشی برای 2024-01-01 18:00:00 — پر کردن با مقدار کوچک مثبت


Exception in thread Thread-5 (_handle_workers):
Traceback (most recent call last):
  File "d:\ProgramData\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "d:\ProgramData\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "d:\ProgramData\anaconda3\Lib\multiprocessing\pool.py", line 516, in _handle_workers
    cls._maintain_pool(ctx, Process, processes, pool, inqueue,
  File "d:\ProgramData\anaconda3\Lib\multiprocessing\pool.py", line 340, in _maintain_pool
    Pool._repopulate_pool_static(ctx, Process, processes, pool,
  File "d:\ProgramData\anaconda3\Lib\multiprocessing\pool.py", line 329, in _repopulate_pool_static
    w.start()
  File "d:\ProgramData\anaconda3\Lib\multiprocessing\process.py", line 121, in start
    self._popen = self._Popen(self)
                  ^^^^^^^^^^^^^^^^^
  File "d:\ProgramData\anaconda3\Lib\multiprocessing\context.py", line 337, in _Popen
    return Popen(process_obj)
      